In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
import json
import os
import re
import time
import pathlib
import annoy
from text_dialog import dialog_dict
from tqdm import tqdm_notebook
from string import punctuation
from spacy.lang.ru.stop_words import STOP_WORDS
from navec import Navec
from slovnet import NER

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

In [4]:
import logging
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# PROJECT INFO

**Тема**: **"Создание прототипа бота-психотераверта"**

Данные взяты с сайта **https://psychoambulanz.ru/** (Лечение панических атак и депрессии)

PS. Для России тема точно актуальная, хотя обработка таких данных не самое приятное занятие 

## Схема обработки данных и реализации  **бота**

![title](process.png)

# LOAD DATA

## NER PERSON PROCESS

Ответы в датасете **персонализированы** - необходимо избавиться от обращений.

In [5]:
skip_step = True

if not skip_step:
    !wget https: // storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar
    !wget https: // storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

In [6]:
skip_step = False

if not skip_step:
    navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
    ner = NER.load('slovnet_ner_news_v1.tar')
    ner.navec(navec)

In [7]:
# замена имен пациентов на "<mtg> Patient <mtg>" - в диалогах будет заменятся на ник обратившегося пользователя

def name_replace(text, ner, author=None):

    text = re.sub('горбатов', 'DOCTOR', text, flags=re.I)
    markup = ner(text)
    name_list = []
    if author:
        name_list.append(author)

    for span in markup.spans:
        if span.type == 'PER':
            name_list.append(text[span.start: span.stop])

    for name in name_list:
        text = text.replace(name, '<mtg> Patient <mtg>')

    return text

In [8]:
test_text = 'Олег Петрович отведал этих мягких французских булочек. Олег счастлив'
print(name_replace(test_text, ner))
print('*'*100)

<mtg> Patient <mtg> отведал этих мягких французских булочек. <mtg> Patient <mtg> счастлив
****************************************************************************************************


## Questions & Answers process and load

В датасете информация собрана в виде диалогов. Думаю стоит разделить на две части:
* 1) Первый вопрос и первый ответ

* 2) Все остальное

Формат telegram-бота плохо подходить длительных консультаций (для этого лучше подходит форум-формат). Поэтому в дальнейшем будем использовать только датафрейм с первичными обращениями пациентов. (Хотя можно было использовать для первого вопроса только ответы из датасета №1 а последующие из №1 и №2 в соответсвии с мерой похожести)

In [9]:
sw = set(list(STOP_WORDS)) - {'не', 'ни', 'нет'}
exclude = set(punctuation)

In [10]:
def preprocess_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text


def preprocess_question(text, sw, exclude):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = preprocess_text(text)
    text = "".join(i for i in text.strip() if i not in exclude).split()
    text = [i.lower() for i in text if i.lower() not in sw and i != ""]
    return " ".join(text)


def preprocess_answer(data, ner):
    data['answer_ready'] = data.apply(lambda row: name_replace(
        row['answer'], ner, row['author']), axis=1)
    return data

Структра диалогов в датастете не всегда представлена в виде **вопрос-ответ**. Бывают случаи, кодга перед ответом идут два вопроса подряд или ответ разбит на несколько частей. Эти разбиения надо слить в один вопрос/ответ соответсвенно. 

Кроме того, в диалогах иногда первый вопрос представлен в виде истории болезни, которая конфиденциальна и **скрыта настройками приватности**. Иными словами ответ есть, а вопроса - нет. Бывают случае, когда вопрос задан, а ответа нет. Такие диалоги будут удальться из итоговых датасетов.

In [11]:
def load_data(json_path):
    frame = {'id': [], 'title': [], 'author': [], 'question': [], 'answer': []}
    df_data1 = pd.DataFrame(frame)  # для первого вопроса и ответа
    df_data2 = pd.DataFrame(frame)  # для остальных

    with open(json_path, "r", encoding="utf-8") as read_file:
        data = json.load(read_file)

    for case in tqdm_notebook(data):
        frame = {}
        question_status = False
        answer_status = False
        first_status = True
        question = ''
        answer = ''

        frame['id'] = case['d_id']
        frame['title'] = case['title']
        frame['author'] = case['dialog']['author'][0]
        dialog_len = len(case['dialog']['author'])

        for num, dialog in enumerate(case['dialog']['text']):

            if (num == 0) and (preprocess_text(dialog) == ''):
                break

            if (num > dialog_len-1):
                break

            if case['dialog']['author'][num] != 'Др_Горбатов':
                question = question + ' ' + dialog

                try:
                    if case['dialog']['author'][num+1] == 'Др_Горбатов':
                        question_status = True
                except IndexError:
                    break

            else:
                answer = answer + ' ' + dialog

                try:
                    if case['dialog']['author'][num+1] != 'Др_Горбатов':
                        answer_status = True
                except IndexError:
                    answer_status = True

            if question_status and answer_status:

                frame['question'] = preprocess_text(question)
                frame['answer'] = preprocess_text(answer)

                if first_status:
                    df_data1 = df_data1.append(frame, ignore_index=True)
                    first_status = False
                else:
                    df_data2 = df_data2.append(frame, ignore_index=True)

                if len(case['dialog']['author']) != len(case['dialog']['text']):
                    break

                question_status = False
                answer_status = False
                question = ''
                answer = ''

    return df_data1, df_data2

In [12]:
json_path_1 = os.path.join(pathlib.Path(os.getcwd()),
                           "data", "psychoambulanz.json")

json_path_2 = os.path.join(pathlib.Path(os.getcwd()),
                           "data", "psychoambulanz_ind.json")

In [13]:
skip_step = True

if not skip_step:
    df1, df2 = load_data(json_path_1)
    df_data_1 = df1
    df_data_2 = df2

    df1, df2 = load_data(json_path_2)
    df_data_1 = df_data_1.append(df1, ignore_index=True)
    df_data_2 = df_data_2.append(df2, ignore_index=True)

    df_data_1.to_csv('df_data_1.csv', index=False)
    df_data_2.to_csv('df_data_2.csv', index=False)

*************************************

In [14]:
df_data_1 = pd.read_csv('df_data_1.csv')  # первые вопрос и ответ
df_data_2 = pd.read_csv('df_data_2.csv')  # остальные вопросы и ответы

In [15]:
df_data_1.iloc[3:5]

,id,title,author,question,answer
3,269126998,Помогите вылечиться,mknew,"Мне 33 года.В 2011 была беременность, которая ...","Чем вы болеете ? Хороший вопрос, но без живой ..."
4,269127481,Гтр,Nata1976,"Добрый день, доктор! Вчера обращалась к вам с ...","Оксана, я недеюсь на то, что леветирацетам в д..."


In [16]:
df_data_1['question_len'] = df_data_1['question'].apply(lambda x: len(x))
df_data_2['question_len'] = df_data_2['question'].apply(lambda x: len(x))
df_data_1['answer_len'] = df_data_1['answer'].apply(lambda x: len(str(x)))
df_data_2['answer_len'] = df_data_2['answer'].apply(lambda x: len(str(x)))

df_data_1 = df_data_1.loc[(df_data_1['question_len'] > 10)
                          & (df_data_1['answer_len'] > 10)]
df_data_2 = df_data_2.loc[(df_data_2['question_len'] > 10)
                          & (df_data_2['answer_len'] > 10)]

df_data_1 = df_data_1.loc[(df_data_1['question_len'] < 10000)
                          & (df_data_1['answer_len'] < 10000)]
df_data_2 = df_data_2.loc[(df_data_2['question_len'] < 10000)
                          & (df_data_2['answer_len'] < 10000)]

print(f'Размер датасетов: 1) - {len(df_data_1)}, 2) - {len(df_data_2)} вопрос-ответов')
print('*'*100)
display(df_data_1[['question_len', 'answer_len']].describe(percentiles=[.5]),
        df_data_2[['question_len', 'answer_len']].describe(percentiles=[.5]))

Размер датасетов: 1) - 16311, 2) - 4159 вопрос-ответов
****************************************************************************************************


,question_len,answer_len
count,16311.00,16311.00
mean,1047.41,370.41
std,1264.55,525.95
min,21.00,11.00
50%,667.00,213.00
max,9996.00,9527.00


,question_len,answer_len
count,4159.00,4159.00
mean,1393.65,1051.35
std,1384.99,935.04
min,15.00,13.00
50%,983.00,776.00
max,9972.00,9049.00


## Final data processing

In [17]:
skip_step = False
GPT = False

if not skip_step:

    # задаем индексы вопросов
    idx = np.arange(99999999, 99999999-len(df_data_1), -1)
    df_data_1['id'] = idx

    idx = np.arange(89999999, 89999999-len(df_data_2), -1)
    df_data_2['id'] = idx

    # Заменяем имена в ответах
    df_data_1 = preprocess_answer(df_data_1, ner)
    df_data_2 = preprocess_answer(df_data_2, ner)

    # Добавляем тему к вопросу
    df_data_1['question'] = df_data_1['title'] + ' ' + df_data_1['question']
    df_data_2['question'] = df_data_2['title'] + ' ' + df_data_2['question']

    # Удаляем стоп-слова и пунктуацию из вопроса
    df_data_1['question_ready'] = df_data_1.apply(
        lambda row: preprocess_question(row['question'], sw, exclude), axis=1)
    df_data_2['question_ready'] = df_data_2.apply(
        lambda row: preprocess_question(row['question'], sw, exclude), axis=1)

    if GPT:
        # добавляем теги начала и конца ответа
        df_data_1['answer_ready'] = '<soa> ' + \
            df_data_1['answer_ready'] + ' <eoa>'
        df_data_2['answer_ready'] = '<soa> ' + \
            df_data_2['answer_ready'] + ' <eoa>'

        # добавяем индекс вопроса в ответ
        def query_add(query, answer):
            text = '<qtg> ' + str(query) + ' <qtg> ' + answer
            return text

        df_data_1['answer_ready'] = df_data_1.apply(
            lambda row: query_add(row['id'], row['answer_ready']), axis=1)
        df_data_2['answer_ready'] = df_data_2.apply(
            lambda row: query_add(row['id'], row['answer_ready']), axis=1)

In [18]:
df_data_1 = df_data_1[['id', 'question_ready', 'answer_ready']]
df_data_2 = df_data_2[['id', 'question_ready', 'answer_ready']]

In [19]:
df_data_1.iloc[3:5]

,id,question_ready,answer_ready
4,99999996,гтр добрый день доктор вчера обращалась вопрос...,"<mtg> Patient <mtg>, я недеюсь на то, что леве..."
5,99999995,тревожное расстройство депрессия здравствуйте ...,1. Достаточна ли доза паксила 1/4 в сутки DOCT...


In [20]:
df_data_1['answer_ready'][15]

'Можно, с сегодня на завтра прекратите приём 37.5 мг венлафаксина и замените его эсциталопрамом в дозе 5 мг - 7 дней, затем доведите его дозу до 10 мг - 2-4 недели и при необходимости, и до 15 мг - 4 недели. Затем выберите из трех, протестированных вами доз эсциталопрама, самую подходящую вам и принимайте её дальше.'

In [21]:
df_data_1.to_csv('df_1_prepared.csv', index=False)
df_data_2.to_csv('df_2_prepared.csv', index=False)

*****************************************

# Get BERT embedings

In [22]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
bert_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [23]:
df_data_pr_1 = pd.read_csv('df_1_prepared.csv')  # первые вопрос и ответ
df_data_pr_2 = pd.read_csv('df_2_prepared.csv')  # остальные вопросы и ответы
df_small_talk = pd.read_csv('small_talk1.csv', delimiter=';', encoding='utf-8')

In [24]:
df_small_talk.head(2)

,Question,Answer
0,Кто ты,"Меня зовут PsyBot, я тестовый вариант бота-пси..."
1,Как настроение,Да клево все!


In [25]:
VEC_LEN = 768

In [26]:
def mean_pooling(model_output, attention_mask):

    # First element of model_output contains all token embeddings
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    return sum_embeddings / sum_mask

In [27]:
def get_embedding(text, sw=sw, exclude=exclude, text_process=True):
    if text_process:
        text = preprocess_question(text, sw, exclude)

    tok = tokenizer(text,
                    return_token_type_ids=False,
                    return_tensors='pt',
                    truncation=True,
                    max_length=512,
                    padding=False
                    )

    with torch.no_grad():
        out_state = bert_model(**tok)

    sentence_embeddings = mean_pooling(out_state, tok['attention_mask'])

    return sentence_embeddings[0]

In [28]:
def get_annoy_index(text_array, sw=sw, exclude=exclude, text_process=True):

    bert_index = annoy.AnnoyIndex(VEC_LEN, 'angular')

    for num, text in enumerate(tqdm_notebook(text_array)):
        out_state = get_embedding(text, sw, exclude, text_process=text_process)
        bert_index.add_item(num, out_state)

    return bert_index

In [29]:
def get_nns(annoy_idx, question, nnn=1):

    emb = get_embedding(question)
    idx, dis = annoy_idx.get_nns_by_vector(emb, nnn, include_distances=True)

    return idx, dis

In [30]:
skip_step = True

if not skip_step:

    small_talk_idx = get_annoy_index(
        df_small_talk['Question'], text_process=True)
    small_talk_idx.build(50)
    small_talk_idx.save('st.annoy')

    data1_idx = get_annoy_index(df_data_pr_1['question_ready'], text_process=False)
    data1_idx.build(50)
    data1_idx.save('data1.annoy')

    data2_idx = get_annoy_index(df_data_pr_1['question_ready'], text_process=False)
    data2_idx.build(50)
    data2_idx.save('data2.annoy')

In [31]:
small_talk_idx = annoy.AnnoyIndex(VEC_LEN, 'angular')
small_talk_idx.load('st.annoy')

data1_idx = annoy.AnnoyIndex(VEC_LEN, 'angular')
data1_idx.load('data1.annoy')

data2_idx = annoy.AnnoyIndex(VEC_LEN, 'angular')
data2_idx.load('data2.annoy')

True

In [32]:
question = 'Как дела'
idx, dis = get_nns(small_talk_idx, question, 1)
print(idx, dis, f'Ответ: {df_small_talk.Answer[idx[0]]}')

[14] [0.0] Ответ: Да клево все!


In [33]:
# не из базы
question = 'Диагноз ТДР. Страдаю ТДР много лет. Очень впячетлительная и ответсвенная.\
Хочется узнать ваше мнение о препарате паксил. Может ли он помочь при депрессии средней тяжести, когда нет сил,\
энергии иногда тоска и все вокруг кажется пасмурным и неинтересным, тревога и иногда ПА (редко ).\
Плохой сон долго немогу уснуть часов до 2 - 3 ночи. Стоит ли мне начинать его принимать или поробовать чтото другое?\
Еще присутствуют сомотофорные симптомы от них еше сильнее тревога'

In [34]:
idx, dis = get_nns(data1_idx, question, 1)
print(idx, dis, f'Ответ:\n{df_data_pr_1.answer_ready[idx[0]]}')

[16066] [0.23478952050209045] Ответ:
Попробуйте, если позволяют средства, перейти на один из антидепрессантов класса SNri ( венлафаксин или дулоксетин ). Это Ад двойного действия и с учетом апатии, сонливости, вялости, повышенной утомляемости, я думаю, что они смогут ВАам помочь лучше, чем паксил.


# Add dialogGPT model

In [35]:
gpt_tokenizer = AutoTokenizer.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
gpt_model = AutoModelForCausalLM.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [36]:
def get_length_param(text: str) -> str:

    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'

    return len_param


def get_gpt_answer(input_user: str, step: int, chat_history_ids=None) -> str:

    new_user_input_ids = gpt_tokenizer.encode(f"|0|{get_length_param(input_user)}|" +
                                              input_user + gpt_tokenizer.eos_token +
                                              "|1|1|", return_tensors="pt")

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat(
        [chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    chat_history_ids = gpt_model.generate(
        bot_input_ids,
        num_return_sequences=1,
        max_length=512,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.6,
        mask_token_id=gpt_tokenizer.mask_token_id,
        eos_token_id=gpt_tokenizer.eos_token_id,
        unk_token_id=gpt_tokenizer.unk_token_id,
        pad_token_id=gpt_tokenizer.pad_token_id,
        device='cpu',
    )

    to_decode = chat_history_ids[:, bot_input_ids.shape[-1]:][0]
    gpt_answer = gpt_tokenizer.decode(to_decode, skip_special_tokens=True)

    return gpt_answer, chat_history_ids

In [37]:
text = "Сколько стоит стог сена"
get_gpt_answer(text, 0)[0]

'В зависимости от количества и качества сена.'

****************************************

# Telegram Bot

![title](bot_logic.png)

Бот **@your_PsyBot** работает в двух режимах:
* **"консультация"** 
* **"просто поговорить"**. 

Для переключения между режимами используются команды(сообщения) **"need help"** и **"just speak"**. Режим "консультация" активирован по умолчанию. После двух неудачных вопросов в режиме "конусультация" (когда соответсвия не находятся) автоматически активируется режим "просто поговорить". Если сообщение содержит менее 10 слов оно будет рассмотрено в режиме "просто поговорить". Все переключения между режимами сопровождаются соответсвющими сообщениями.

Для ответов на базовые вопросы (кто ты? что умеешь...) используется данные датафрейма small_talk и все вопросы изначально проходят через него.

In [38]:
DIS_ST = 0.5  # расстояние для выбора ответов в small talk
DIS_PSY = 0.27  # расстояние для выбора ответов в режиме консультация
MAX_Q = 2  # максимальное количество попыток задать вопрос консультанту до перехода в режим "просто поговорить"
MIN_QL = 10  # минимальное число слов для рассмотрения вопроса в режиме консультации

In [39]:
# Enable logging
logging.basicConfig(filename='bot.log',
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

logger = logging.getLogger()

In [40]:
TOKEN = '===========TOKEN================'  # Токен API к Telegram
updater = Updater(TOKEN, use_context=True)
dispatcher = updater.dispatcher

In [41]:
def start(update, context):
    
    context.chat_data['need_help'] = True
    context.chat_data['num_question'] = 0
    context.chat_data['step'] = 0
    context.chat_data['his_tensor'] = None
    name = update.message.chat.first_name
    
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=f"Привет, {name}! Я - PSYBOT или просто Доктор:)")
    text_help = dialog_dict['text_help']
    for text in text_help:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)


def response(update, context):
    
    name = update['message']['chat']['first_name']

    if not context.chat_data:
        context.chat_data['need_help'] = True
        context.chat_data['num_question'] = 0
        context.chat_data['step'] = 0
        context.chat_data['his_tensor'] = None

    answer_ready = False
    question = update.message.text
    question = preprocess_text(question)
    que_len = len([word for word in re.findall(r'\w+', question)])
    prep_question = preprocess_question(question, sw=sw, exclude=exclude)

    if question.lower() == "speak with me":
        answer = dialog_dict['speak_with_me']
        context.chat_data['need_help'] = False
        context.chat_data['num_question'] = 0
        answer_ready = True

    if question.lower() == "need help":
        answer = dialog_dict['need_help']
        context.chat_data['need_help'] = True
        context.chat_data['num_question'] = 0
        answer_ready = True

    if not answer_ready:
        idx, dis = get_nns(small_talk_idx, prep_question, 1)

        if dis[0] <= DIS_ST:
            answer = df_small_talk['Answer'][idx[0]]
            answer_ready = True
        else:
            if context.chat_data['need_help'] and (que_len >= MIN_QL):
                idx, dis = get_nns(data1_idx, prep_question, 1)
                if dis[0] <= DIS_PSY:
                    context.chat_data['num_question'] = 0
                    answer = df_data_pr_1['answer_ready'][idx[0]]
                    answer = answer.replace('<mtg> Patient <mtg>', name)
                    answer_ready = True
                else:
                    context.chat_data['num_question'] += 1
                    if context.chat_data['num_question'] < MAX_Q:
                        answer = dialog_dict['not_found']
                    else:
                        context.chat_data['need_help'] = False
                        context.chat_data['num_question'] = 0
                        answer = dialog_dict['sorry'] + dialog_dict['speak_with_me']
                        context.bot.send_message(chat_id=update.effective_chat.id,
                                                 text=answer)

        if (not context.chat_data['need_help'] or (que_len < MIN_QL)) and not answer_ready:
            answer, context.chat_data['his_tensor'] = get_gpt_answer(question,
                                                                     context.chat_data['step'],
                                                                     context.chat_data['his_tensor'])
            context.chat_data['step'] += 1
            
    info = name + '/ ' + question + '/ ' + answer
    logger.info(info)
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=answer)

In [42]:
start_handler = CommandHandler(['start', 'help'], start)
answer_handler = MessageHandler(Filters.text & (~Filters.command), response)

dispatcher.add_handler(start_handler)
dispatcher.add_handler(answer_handler)

In [43]:
updater.start_polling()
updater.idle()

# RESULTS

In [44]:
from IPython.display import HTML, display

## Small talk + GPT

In [45]:
display(HTML("<table><tr><td><img src='./results/sm1.jpg'></td><td><img src='./results/sm2.jpg'></td></tr></table>"))

,


## 	Consultation mode

In [46]:
display(HTML("<table><tr><td><img src='./results/consulting1.jpg'></td><td><img src='./results/consulting2.jpg'></td></tr></table>"))

,


In [47]:
display(HTML("<table><tr><td><img src='./results/consulting3.jpg'></td><td><img src='./results/consulting4.jpg'></td></tr></table>"))

,


PS. Бота нельзя выпускать в сеть. Может посоветовать чет не то.